In [1]:
import numpy as np
import pandas as pd

In [2]:
# 데이터셋 : 타이타닉 탑승객 데이터 (출처 : https://www.kaggle.com/)
train = pd.read_csv('https://bit.ly/fc-ml-titanic') 

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 전처리 : train / validation 세트 나누기
1. feature 와 label을 정의합니다.
2. 적절한 비율로 train / validation set을 나눕니다.

In [4]:
feature = [
    'Pclass', 'Sex', 'Age', 'Fare'
]

In [5]:
label = [
    'Survived'
]

In [6]:
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [7]:
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [8]:
from sklearn.model_selection import train_test_split

- test_size : validation set에 할당할 비율(20% -> 0.2)
- shuffle : 셔플옵션 (기본 True)
- random_state : 랜덤 시드값

In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, shuffle=True, random_state=30)

In [10]:
x_train.shape, y_train.shape

((712, 4), (712, 1))

In [11]:
x_valid.shape, y_valid.shape

((179, 4), (179, 1))

### 전처리 : 결측치

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


#### 결측치를 확인하는 방법은 pandas의 isnull()
#### 합계를 구하는 sum()을 통해 확인할 수 있습니다.

In [13]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 개별 컬럼의 결측치에 대하여 확인하는 방법

In [14]:
train['Age'].isnull().sum()

177

### 수치형 데이터에 대한 결측치 처리

In [15]:
train['Age'].fillna(0).describe() # 결측치에 0 넣기

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [16]:
train['Age'].fillna(train['Age'].mean()).describe() # 결측치에 평균을 넣기

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

#### imputer : 2개 이상의 컬럼을 한번에 처리할 때

In [19]:
from sklearn.impute import SimpleImputer

In [20]:
imputer = SimpleImputer(strategy = 'mean')

In [21]:
imputer.fit(train[['Age', 'Pclass']])

SimpleImputer()

In [22]:
result = imputer.transform(train[['Age', 'Pclass']])

In [23]:
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [24]:
train[['Age','Pclass']] = result

In [25]:
train[['Age','Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [26]:
train[['Age','Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


### fit_transform()은 fit와 transform을 한번에 해주는 함수

In [27]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic') 

In [28]:
train[['Age','Pclass']].isnull().sum()

Age       177
Pclass      0
dtype: int64

In [29]:
imputer = SimpleImputer(strategy='median')

In [30]:
result = imputer.fit_transform(train[['Age','Pclass']])

In [31]:
train[['Age','Pclass']] = result

In [32]:
train[['Age','Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [33]:
train[['Age','Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.361582,2.308642
std,13.019697,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,28.000000,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


### Categorical Column 데이터에 대한 결측치 처리

In [34]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic') 

#### 1개의 column을 처리하는 경우

In [35]:
train['Embarked'].fillna('S')

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

#### Imputer를 사용하는 경우 : 2개 이상의 column을 처리할때

In [36]:
imputer = SimpleImputer(strategy = 'most_frequent')

In [37]:
result = imputer.fit_transform(train[['Embarked', 'Cabin']])

In [38]:
train[['Embarked','Cabin']] = result

In [39]:
train[['Embarked','Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

### Label Encoding : 문자(categorical)를 수치(numerical)로 변환
- 학습을 위해서 모든 문자로된 데이터는 수치로 변환해야 합니다.

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [41]:
def convert(data):
    if data == 'male':
        return 1
    elif data == 'female':
        return 0

In [42]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [43]:
train['Sex'].apply(convert)

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [44]:
from sklearn.preprocessing import LabelEncoder

In [45]:
le = LabelEncoder()

In [46]:
train['Sex_num'] = le.fit_transform(train['Sex'])

In [47]:
train['Sex_num'].value_counts()

1    577
0    314
Name: Sex_num, dtype: int64

In [48]:
le.classes_

array(['female', 'male'], dtype=object)

In [49]:
le.inverse_transform([0,1,1,0,0,1,1])

array(['female', 'male', 'male', 'female', 'female', 'male', 'male'],
      dtype=object)

In [50]:
le.fit_transform(train['Embarked'])

array([2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0,
       2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1,

### 원 핫 인코딩 (One Hot Encoding)

In [51]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic') 

In [52]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [53]:
train['Embarked']=train['Embarked'].fillna('S')

In [54]:
train['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [55]:
train['Embarked_num'] = LabelEncoder().fit_transform(train['Embarked'])

In [56]:
train['Embarked_num'].value_counts()

2    646
0    168
1     77
Name: Embarked_num, dtype: int64

#### Embarked는 탑승항구의 이니셜을 나타냅니다. Laebel Encoder를 통해서 수치형으로 변환했습니다. 하지만 이대로 데이터를 기계학습을 시키면, 기계는 데이터 안에서 관계를 학습합니다.
- 'S' = 2, 'Q' = 1 -> Q + Q = S 가 된다라고 학습.
- 독립적인 데이터는 별도의 컬럼으로 분리하고, 각각의 컬럼에 해당 값에만 True, 나머지는 False를 갖습니다. -> 원 핫 인코딩 

In [57]:
train['Embarked'][:6]

0    S
1    C
2    S
3    S
4    S
5    Q
Name: Embarked, dtype: object

In [58]:
train['Embarked_num'][:6]

0    2
1    0
2    2
3    2
4    2
5    1
Name: Embarked_num, dtype: int32

In [59]:
pd.get_dummies(train['Embarked_num'][:6])

,0,1,2
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


In [60]:
one_hot = pd.get_dummies(train['Embarked_num'][:6])

In [61]:
one_hot.columns = ['C','Q','S']
one_hot

,C,Q,S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0


#### column을 분리시켜 카테고리형 -> 수치형으로 변환하면서 생기는 수치형 값의 관계를 끊어주어서 독립적인 형태로 바꿔줍니다.

#### 원 핫 인코딩은 카테고리(항구,계절,성별...)의 특성을 가지는 column에 대해서 적용합니다.